<a href="https://colab.research.google.com/github/2025-01-sookmyung-opensource/final_project/blob/pre-min/MobileNetV3_amount_estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torchvision.models as models
mobilenet = models.mobilenet_v3_small(pretrained=True)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100%|██████████| 9.83M/9.83M [00:00<00:00, 131MB/s]


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import os

# 데이터 경로
data_dir = "/content/drive/MyDrive/image-20250617T154733Z-1-001"

# 잘 올라갔는지 확인
print("클래스 디렉토리들:", os.listdir(data_dir))


클래스 디렉토리들: ['image']


In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # MobileNetV3 input size
    transforms.ToTensor(),
])

# 데이터셋 불러오기
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# 데이터로더
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 클래스 확인
print("클래스 라벨:", dataset.classes)


클래스 라벨: ['image']


In [5]:
import torchvision.models as models
import torch.nn as nn

model = models.mobilenet_v3_small(pretrained=True)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, len(dataset.classes))


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 손실 함수와 최적화기
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [9]:
num_epochs = 2  # 원하는 만큼 조정

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 통계
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"[Epoch {epoch+1}] Loss: {running_loss:.3f} | Accuracy: {acc:.2f}%")


[Epoch 1] Loss: 0.000 | Accuracy: 100.00%
[Epoch 2] Loss: 0.000 | Accuracy: 100.00%
